In [3]:
import pandas as pd
from sklearn.linear_model import Perceptron
from mlxtend.plotting import plot_decision_regions
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,Flatten,MaxPooling2D,AveragePooling2D
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping
import nltk
nltk.download('stopwords')
import re
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df=pd.read_csv('/content/a1_IMDB_Dataset.csv')


In [5]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')

import string
string.punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()


In [8]:
def transform_text(text):
  #To transform the text into lower characters
  text=text.lower()
  #To seperates the words in a given text
  text=nltk.word_tokenize(text)

  y=[]
  for i in text:
    if i.isalnum():
      y.append(i)
  #we clone the text cuz if we append without cloning the text so the y.clear function will clear the text too
  text=y[:]
  y.clear()
 #to remove stop words and punctuation
  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      y.append(i)

  text=y[:]
  y.clear()
  #to converts words into their root form eg: danacing to dance
  for i in text:
    y.append(ps.stem(i))

  #to make it a string we use " ".join()
  return" ".join(y)


In [9]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
x = []
sentences = list(df['review'])
for text in sentences:
    x.append(transform_text(text))

In [11]:
y=df['sentiment']
y=np.array(list(map(lambda x: 1 if x=='positive' else 0,y)))

In [12]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [13]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(x_train)

x_train = word_tokenizer.texts_to_sequences(x_train)
x_test = word_tokenizer.texts_to_sequences(x_test)


In [14]:
vocablen=len(word_tokenizer.word_index)+1

In [16]:
maxlen=100
x_train=pad_sequences(x_train,padding='post',maxlen=maxlen)
x_test=pad_sequences(x_test,padding='post',maxlen=maxlen)


In [17]:
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('a2_glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [19]:
embedding_matrix = zeros((vocablen, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [27]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.5 MB/s eta 0:00:00


In [28]:
from tensorflow.keras.callbacks import EarlyStopping
import kerastuner as kt

<ipython-input-28-7086c05bff61>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [96]:
# Neural Network architecture

lstm_model = Sequential()
embedding_layer = Embedding(vocablen, 100, input_length=maxlen )

lstm_model.add(embedding_layer)
lstm_model.add(LSTM(128))

lstm_model.add(Dense(1, activation='sigmoid'))

In [97]:
# Model compiling

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(lstm_model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          2954600   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3071977 (11.72 MB)
Trainable params: 3071977 (11.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
callback = EarlyStopping(
     monitor="val_loss",
     min_delta=0.00001,
     patience=20,
     verbose=1,
     mode="auto",
     baseline=None,
     restore_best_weights=False
 )

In [106]:
x_train.shape

(7309, 100)

In [113]:
# Model Training

lstm_model_history = lstm_model.fit(x_train, y_train, batch_size=128, epochs=6, callbacks=callback, validation_split=0.2)

Epoch 1/6
46/46 [==============================] - 1s 19ms/step - loss: 0.0474 - acc: 0.9908 - val_loss: 0.1355 - val_acc: 0.9610
Epoch 2/6
46/46 [==============================] - 1s 28ms/step - loss: 0.0330 - acc: 0.9933 - val_loss: 0.1681 - val_acc: 0.9583
Epoch 3/6
46/46 [==============================] - 1s 21ms/step - loss: 0.0297 - acc: 0.9937 - val_loss: 0.1682 - val_acc: 0.9603
Epoch 4/6
46/46 [==============================] - 1s 26ms/step - loss: 0.0246 - acc: 0.9947 - val_loss: 0.1685 - val_acc: 0.9583
Epoch 5/6
46/46 [==============================] - 1s 19ms/step - loss: 0.0270 - acc: 0.9930 - val_loss: 0.1984 - val_acc: 0.9583
Epoch 6/6
46/46 [==============================] - 2s 37ms/step - loss: 0.0164 - acc: 0.9968 - val_loss: 0.1917 - val_acc: 0.9590


In [114]:
# Predictions on the Test Set

score = lstm_model.evaluate(x_test, y_test)

58/58 [==============================] - 0s 5ms/step - loss: 0.7528 - acc: 0.8337


In [115]:
# Model Performance

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.75276780128479
Test Accuracy: 0.833698034286499


In [118]:
sample_reviews = pd.read_csv("a3_IMDb_Unseen_Reviews.csv")

sample_reviews.head(6)

,Unnamed: 0,Movie,Review Text,IMDb Rating
0,0,Ex Machina,Intelligent Movie.\nThis movie is obviously al...,9
1,1,Ex Machina,Extraordinary and thought-provoking.\n'Ex mach...,10
2,2,Ex Machina,"Poor story, only reasonable otherwise.\nIf I h...",3
3,3,Ex Machina,Had Great Potential.\nThis movie is one of the...,1
4,4,Eternals,Amazing visuals and philosophical concepts!\n\...,10
5,5,Eternals,Worst MCU film ever\n\nFollowing the events of...,3


In [120]:
# Preprocess review text with earlier defined preprocess_text function

unseen_reviews = sample_reviews['Review Text']

unseen_processed = []
for review in unseen_reviews:
  review = transform_text(review)
  unseen_processed.append(review)

In [121]:
# Tokenising instance with earlier trained tokeniser
unseen_tokenized = word_tokenizer.texts_to_sequences(unseen_processed)

# Pooling instance to have maxlength of 100 tokens
unseen_padded = pad_sequences(unseen_tokenized, padding='post', maxlen=maxlen)

In [122]:
# Passing tokenised instance to the LSTM model for predictions
unseen_sentiments = lstm_model.predict(unseen_padded)

unseen_sentiments

1/1 [==============================] - 1s 556ms/step


array([[0.9969664 ],
       [0.78195006],
       [0.0056249 ],
       [0.9364411 ],
       [0.99845445],
       [0.00436249]], dtype=float32)

In [123]:
# Writing model output file back to Google Drive

sample_reviews['Predicted Sentiments'] = np.round(unseen_sentiments*10,1)

df_prediction_sentiments = pd.DataFrame(sample_reviews['Predicted Sentiments'], columns = ['Predicted Sentiments'])
df_movie                 = pd.DataFrame(sample_reviews['Movie'], columns = ['Movie'])
df_review_text           = pd.DataFrame(sample_reviews['Review Text'], columns = ['Review Text'])
df_imdb_rating           = pd.DataFrame(sample_reviews['IMDb Rating'], columns = ['IMDb Rating'])


dfx=pd.concat([df_movie, df_review_text, df_imdb_rating, df_prediction_sentiments], axis=1)

dfx.to_csv("./c2_IMDb_Unseen_Predictions.csv", sep=',', encoding='UTF-8')

dfx.head(6)

,Movie,Review Text,IMDb Rating,Predicted Sentiments
0,Ex Machina,Intelligent Movie.\nThis movie is obviously al...,9,10.0
1,Ex Machina,Extraordinary and thought-provoking.\n'Ex mach...,10,7.8
2,Ex Machina,"Poor story, only reasonable otherwise.\nIf I h...",3,0.1
3,Ex Machina,Had Great Potential.\nThis movie is one of the...,1,9.4
4,Eternals,Amazing visuals and philosophical concepts!\n\...,10,10.0
5,Eternals,Worst MCU film ever\n\nFollowing the events of...,3,0.0
